이번 챕터에서는 파이토치(PyTorch)로 인공 신경망을 이용하여 태깅 작업을 하는 모델을 만듭니다. 개체명 인식기와 품사 태거를 만드는데, 이러한 두 작업의 공통점은 RNN의 다-대-다(Many-to-Many) 작업이면서 또한 앞, 뒤 시점의 입력을 모두 참고하는 양방향 RNN(Bidirectional RNN)을 사용한다는 점입니다.

두 개의 실습 챕터를 진행하기 전에 전체적으로 실습이 어떻게 진행되는지 정리해보도록 하겠습니다. 텍스트 분류 개요 챕터와 겹치는 부분에 대해서는 요약하여 설명하므로, 이해가 되지 않는 부분이 있다면 해당 챕터를 참고바랍니다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.legacy import data,datasets
import time
import random

In [2]:
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# 3개의 필드 정의
TEXT = data.Field(lower = True)
UD_TAGS = data.Field(unk_token = None)
PTB_TAGS = data.Field(unk_token = None)

fields = (("text", TEXT), ("udtags", UD_TAGS), ("ptbtags", PTB_TAGS))

In [4]:
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

In [5]:
print(f"훈련 샘플의 개수 : {len(train_data)}")
print(f"검증 샘플의 개수 : {len(valid_data)}")
print(f"테스트 샘플의 개수 : {len(test_data)}")

훈련 샘플의 개수 : 12543
검증 샘플의 개수 : 2002
테스트 샘플의 개수 : 2077


In [6]:
# 훈련 데이터의 3개의 필드 확인
print(train_data.fields)

{'text': <torchtext.legacy.data.field.Field object at 0x0000022867D54EB0>, 'udtags': <torchtext.legacy.data.field.Field object at 0x0000022868DCAF10>, 'ptbtags': <torchtext.legacy.data.field.Field object at 0x0000022867D54E50>}


In [7]:
# 첫번째 훈련 샘플의 text 필드
print(vars(train_data.examples[0])['text'])

['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.']


In [8]:
# 첫번째 훈련 샘플의 udtags 필드 - 사용할 레이블
print(vars(train_data.examples[0])['udtags'])

['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']


In [9]:
# 첫번째 훈련 샘플의 ptbdtags 필드 - 여기서는 사용안함
print(vars(train_data.examples[0])['ptbtags'])

['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']


In [10]:
# 최소 허용 빈도
MIN_FREQ = 5

# 사전 훈련된 워드 임베딩 GloVe 다운로드
TEXT.build_vocab(train_data, min_freq = MIN_FREQ, vectors = "glove.6B.100d")
UD_TAGS.build_vocab(train_data)
PTB_TAGS.build_vocab(train_data)

In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 9076), ('.', 8640), (',', 7021), ('to', 5137), ('and', 5002), ('a', 3782), ('of', 3622), ('i', 3379), ('in', 3112), ('is', 2239), ('you', 2156), ('that', 2036), ('it', 1850), ('for', 1842), ('-', 1426), ('have', 1359), ('"', 1296), ('on', 1273), ('was', 1244), ('with', 1216)]


In [12]:
# 상위 정수 인덱스 단어 10개 출력
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', '.', ',', 'to', 'and', 'a', 'of', 'i']


In [13]:
# 상위 빈도순으로 udtags 출력
print(UD_TAGS.vocab.freqs.most_common())

[('NOUN', 34781), ('PUNCT', 23679), ('VERB', 23081), ('PRON', 18577), ('ADP', 17638), ('DET', 16285), ('PROPN', 12946), ('ADJ', 12477), ('AUX', 12343), ('ADV', 10548), ('CCONJ', 6707), ('PART', 5567), ('NUM', 3999), ('SCONJ', 3843), ('X', 847), ('INTJ', 688), ('SYM', 599)]


In [14]:
# 상위 정수 인덱스 순으로 출력
print(UD_TAGS.vocab.itos)

['<pad>', 'NOUN', 'PUNCT', 'VERB', 'PRON', 'ADP', 'DET', 'PROPN', 'ADJ', 'AUX', 'ADV', 'CCONJ', 'PART', 'NUM', 'SCONJ', 'X', 'INTJ', 'SYM']


In [15]:
def tag_percentage(tag_counts): # 태그 레이블의 분포를 확인하는 함수
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]

    return tag_counts_percentages

In [16]:
print("Tag  Occurences Percentage\n")
for tag, count, percent in tag_percentage(UD_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag  Occurences Percentage

NOUN	34781	17.0%
PUNCT	23679	11.6%
VERB	23081	11.3%
PRON	18577	 9.1%
ADP	17638	 8.6%
DET	16285	 8.0%
PROPN	12946	 6.3%
ADJ	12477	 6.1%
AUX	12343	 6.0%
ADV	10548	 5.2%
CCONJ	6707	 3.3%
PART	5567	 2.7%
NUM	3999	 2.0%
SCONJ	3843	 1.9%
X	847	 0.4%
INTJ	688	 0.3%
SYM	599	 0.3%


In [17]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

첫 번째 미니배치만 꺼내서 확인

In [18]:
batch = next(iter(train_iterator))

In [19]:
batch


[torchtext.legacy.data.batch.Batch of size 64 from UDPOS]
	[.text]:[torch.cuda.LongTensor of size 46x64 (GPU 0)]
	[.udtags]:[torch.cuda.LongTensor of size 46x64 (GPU 0)]
	[.ptbtags]:[torch.cuda.LongTensor of size 46x64 (GPU 0)]

In [20]:
batch.text.shape

torch.Size([46, 64])

In [21]:
batch.text

tensor([[ 732,  167,    2,  ...,    2,   59,  668],
        [  16,  196,  133,  ..., 2991,   46,    1],
        [   1,   29,   48,  ..., 1582,   12,    1],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')

In [22]:
# 이번 모델에서는 batch_first=True를 사용하지 않으므로 배치 차원이 맨 앞이 아님.
class RNNPOSTagger(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout): 
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional = bidirectional)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)        
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))

        # embedded = [sent len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(embedded)

        # output = [sent len, batch size, hid dim * n directions]
        # hidden/cell = [n layers * n directions, batch size, hid dim]
        predictions = self.fc(self.dropout(outputs))

        # predictions = [sent len, batch size, output dim]
        return predictions

In [23]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(UD_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = RNNPOSTagger(INPUT_DIM, 
                     EMBEDDING_DIM, 
                     HIDDEN_DIM, 
                     OUTPUT_DIM, 
                     N_LAYERS, 
                     BIDIRECTIONAL, 
                     DROPOUT)

In [24]:
# 파라미터 개수 출력
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,027,510 trainable parameters


In [25]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([3921, 100])


In [26]:
model.embedding.weight.data.copy_(pretrained_embeddings) # 임베딩 벡터값 copy

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1020,  0.7700,  0.1169,  ..., -0.1416, -0.1932, -0.4225],
        [-0.0263,  0.0179, -0.5016,  ..., -0.8688,  0.9409, -0.2882],
        [ 0.1519,  0.4712,  0.0895,  ..., -0.4702, -0.3127,  0.1078]])

In [27]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
print(UNK_IDX)
print(PAD_IDX)

0
1


In [28]:
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM) # 0번 임베딩 벡터에는 0값을 채운다.
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM) # 1번 임베딩 벡터에는 1값을 채운다.
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1020,  0.7700,  0.1169,  ..., -0.1416, -0.1932, -0.4225],
        [-0.0263,  0.0179, -0.5016,  ..., -0.8688,  0.9409, -0.2882],
        [ 0.1519,  0.4712,  0.0895,  ..., -0.4702, -0.3127,  0.1078]])


## 5. 옵티마이저와 비용 함수 구현하기

In [29]:
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]
print(TAG_PAD_IDX)

0


In [30]:
optimizer = optim.Adam(model.parameters())

In [31]:
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [32]:

model = model.to(device)
criterion = criterion.to(device)


In [33]:
prediction = model(batch.text)

In [34]:
prediction.shape

torch.Size([46, 64, 18])

In [35]:
prediction = prediction.view(-1, prediction.shape[-1])
prediction.shape

torch.Size([2944, 18])

In [36]:
batch.udtags.shape

torch.Size([46, 64])

In [37]:
batch.udtags.view(-1).shape

torch.Size([2944])

## 훈련과 평가

    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).cuda() -> cuda로 해줘야 cpu로 안돌아간다

In [44]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    미니 배치에 대한 정확도 출력
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).cuda()

In [45]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        text = batch.text
        tags = batch.udtags

        optimizer.zero_grad()

        #text = [sent len, batch size]     
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        predictions = predictions.view(-1, predictions.shape[-1]) # #predictions = [sent len * batch size, output dim]
        tags = tags.view(-1) # tags = [sent len * batch_size]

        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        loss = criterion(predictions, tags)

        acc = categorical_accuracy(predictions, tags, tag_pad_idx)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [46]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text = batch.text
            tags = batch.udtags

            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            loss = criterion(predictions, tags)

            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [47]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01
	Train Loss: 1.085 | Train Acc: 66.28%
	 Val. Loss: 0.670 |  Val. Acc: 79.17%
Epoch: 02
	Train Loss: 0.384 | Train Acc: 87.80%
	 Val. Loss: 0.535 |  Val. Acc: 83.36%
Epoch: 03
	Train Loss: 0.302 | Train Acc: 90.25%
	 Val. Loss: 0.496 |  Val. Acc: 84.34%
Epoch: 04
	Train Loss: 0.263 | Train Acc: 91.47%
	 Val. Loss: 0.456 |  Val. Acc: 85.46%
Epoch: 05
	Train Loss: 0.239 | Train Acc: 92.24%
	 Val. Loss: 0.438 |  Val. Acc: 85.76%
Epoch: 06
	Train Loss: 0.219 | Train Acc: 92.81%
	 Val. Loss: 0.429 |  Val. Acc: 85.95%
Epoch: 07
	Train Loss: 0.203 | Train Acc: 93.26%
	 Val. Loss: 0.437 |  Val. Acc: 85.76%
Epoch: 08
	Train Loss: 0.190 | Train Acc: 93.69%
	 Val. Loss: 0.413 |  Val. Acc: 86.25%
Epoch: 09
	Train Loss: 0.179 | Train Acc: 94.00%
	 Val. Loss: 0.417 |  Val. Acc: 86.29%
Epoch: 10
	Train Loss: 0.167 | Train Acc: 94.43%
	 Val. Loss: 0.409 |  Val. Acc: 86.67%


In [48]:
test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.414 |  Test Acc: 87.07%
